<a href="https://colab.research.google.com/github/tomvangasse/game-store-manager/blob/main/tomsMenuProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Upload files here
Click run and then click choose files:

    Video_Game_Info.txt
    Board_Game_Info.txt
    Subscription_Info.txt
    Game_feedback.txt
    Booking.txt
    Rental.txt
    feedbackManager.pyc
    subscriptionManager.pyc
'''


from google.colab import files
uploaded = files.upload()

In [ ]:
import subscriptionManager as smTV
import feedbackManager as fbTV

sub = smTV.load_subscriptions()

print(sub)


Register / login

In [ ]:
def checkSubscription(userID):
    while True:
        subscriptions = smTV.load_subscriptions()
        if smTV.check_subscription(userID, subscriptions):
            return userID
        else:
            login(userID)

    subType = subscriptions[userID]["SubscriptionType"]


def register(userID):
    import subscriptionManager as smTV
    import datetime
    from dateutil.relativedelta import relativedelta as rd

    print("Unregistered account, please register now:")
    while True:
        subscriptionType = input("Enter subscription type (Basic or premium): ").lower()
        if subscriptionType != "basic" and subscriptionType != "premium":
            print("Invalid subscription type")
        else:
            break
    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    endTime = (date + rd(years=3)).strftime("%Y-%m-%d")

    with open("Subscription_Info.txt", mode = "a") as file:
        file.write(f"\n{userID},{subscriptionType},{startTime},{endTime}")

    print(f"Logged in as {userID}")
    return subscriptionType


def login(userID):
    import subscriptionManager as smTV

    subscriptions = smTV.load_subscriptions()

    if smTV.check_subscription(userID, subscriptions):
        print(f"Logged in as {userID}")

    else:
        register(userID)

def checkUserLength(userID):
    if len(userID) != 4:
        return False
    else:
        return True

Game search

In [ ]:
import csv

def getVideoDict():
    with open("Video_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def getBoardDict():
    with open("Board_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def printGames(gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        for i in boardDict:
            currentGame = boardDict[i]
            print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        print("\n")
    elif gameType == "v":
        for i in videoDict:
            currentGame = videoDict[i]
            print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        print("\n")
    else:
        return

def printGameInfo(GameID, gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        gameInfo = boardDict[GameID]
        for key, value in gameInfo.items():
            print(f"{key: >20}:  {value}")
        print("\n")
    elif gameType == "v":
        gameInfo = videoDict[GameID]
        for key, value in gameInfo.items():
            print(f"{key: >20}:  {value}")
        print("\n")
    else:
        return


def gameSearch():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    gameType = input('''Board games (b)
Video games (v)
Quit (q)
''')

    if gameType == "q":
        return False

    gameSelection = input('''Print games (press enter)
Select a specific game (enter GameID)
''')
    gameSelection
    if gameSelection == "":
        printGames(gameType)
    elif gameSelection == "q":
        return False
    else:
        printGameInfo(gameSelection, gameType)

while True:
    if gameSearch() == False:
        break


Update text files:

In [ ]:
def updateVideo(videoDict):
    headers = ["GameID", "Game name", "Platform", "Genre", "Date added", "Available copies", "Unavailable copies"]
    writer = csv.DictWriter(open("Video_Game_Info.txt", mode="w", newline=""), fieldnames=headers)
    writer.writeheader()
    for key, value in videoDict.items():
        writer.writerow(value)
    return

def updateBoard(boardDict):
    headers = ["GameID", "Game name", "No of players", "Genre", "Date added", "Available copies", "Unavailable copies"]
    writer = csv.DictWriter(open("Board_Game_Info.txt", mode="w", newline=""), fieldnames=headers)
    writer.writeheader()
    for key, value in boardDict.items():
        writer.writerow(value)
    return

def updateRental(rentalDict):
    p



Game rent

In [25]:
import subscriptionManager as smTV
import feedbackManager as fbTV


def rentBoardGame(GameID):
    boardDict = getBoardDict()
    availableCopies = boardDict[GameID]["Available copies"]
    unavailableCopies = boardDict[GameID]["Unavailable copies"]
    boardDict[GameID]["Available copies"] = str(int(availableCopies) - 1)
    boardDict[GameID]["Unavailable copies"] = str(int(unavailableCopies) + 1)
    updateBoard(boardDict)

def rentVideoGame(GameID):
    videoDict = getVideoDict()
    availableCopies = videoDict[GameID]["Available copies"]
    unavailableCopies = videoDict[GameID]["Unavailable copies"]
    videoDict[GameID]["Available copies"] = str(int(availableCopies) - 1)
    videoDict[GameID]["Unavailable copies"] = str(int(unavailableCopies) + 1)
    updateVideo(videoDict)

def updateRentGame(gameID, userID):
    from dateutil.relativedelta import relativedelta as rd
    import datetime

    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    with open("Rental.txt", mode="a", newline="") as file:
        file.write(f"\n{userID},{gameID},{startTime},")


def checkAvailability(gameID, gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        currentGame = boardDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True

    elif gameType == "v":
        currentGame = videoDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True
    else:
        return


def rentGameMain():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    while True:
        userID = input("Enter user ID: ")
        if checkUserLength(userID):
            userID = checkSubscription(userID)
            break
        else:
            print("Invalid user ID")


    selectedGameID = input('''Enter the GameID for the desired game:
''')
    if selectedGameID in videoDict:
        if checkAvailability(selectedGameID, "v"):
            rentVideoGame(selectedGameID)
            updateRentGame(selectedGameID, userID)
            currentGame = videoDict[selectedGameID]
            print(f"{currentGame["Game name"]} has been rented by {userID}")

        else:
            print("Game not available")

    elif selectedGameID in boardDict:
        print(checkAvailability(selectedGameID, "b"))
        if checkAvailability(selectedGameID, "b"):
            rentBoardGame(selectedGameID)
            updateRentGame(selectedGameID, userID)
            currentGame = boardDict[selectedGameID]
            print(f"{currentGame["Game name"]} has been rented by {userID}")

        else:
            print("Game not available")

    else:
        print("Invalid GameID")



#with open("Rental.txt", mode="a") as file:
    #file.write(f"\n{gameID},{Rental date}

rentGameMain()

Enter user ID: tomv
Enter the GameID for the desired game:
risk01b
True
Risk has been rented by tomv


Returns


In [40]:
def getRentalList():
    with open("Rental.txt", mode="r", newline="") as file:
        count = 0
        rentalList = []
        for line in file:
            if count == 0:
                count += 1
                continue
            else:
                currentLine = line.rstrip().split(",")
                print(currentLine)
            count += 1
            rentalList.append(currentLine)
    return rentalList

def getUserRental(userID, rentalList):
    userRentals = []
    for row in rentalList:
        if row[0] == userID:
            userRentals.append(row)
    return userRentals



def returnGameMain():
    rentalList = getRentalList()
    while True:
        userID = input("Enter user ID: ")
        if checkUserLength(userID):
            userID = checkSubscription(userID)
            break
        else:
            print("Invalid user ID")
    print(getUserRental(userID, rentalList))

returnGameMain()


['abcd', 'tloz01', '2025-10-10', '25-11-10']
['tomv', 'fzhz01v', '2025-11-06', '']
['tomv', 'risk01b', '2025-11-06', '']
Enter user ID: tomv
[['tomv', 'fzhz01v', '2025-11-06', ''], ['tomv', 'risk01b', '2025-11-06', '']]
